In [50]:
import json
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [51]:
path = 'data/kaggle/'
data = pd.read_csv(path + 'train.csv',
                   parse_dates=['Date'],
                   dtype={
                       'Id': 'int',
                       'Province_State': 'category',
                       'Country_Region': 'category',
                       'ConfirmedCases': 'int',
                       'Fatalities': 'int'
                   })
data.replace([np.inf, -np.inf], np.nan)
print(data.shape)
data

(22644, 6)


,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0,0
1,2,NaN,Afghanistan,2020-01-23,0,0
2,3,NaN,Afghanistan,2020-01-24,0,0
3,4,NaN,Afghanistan,2020-01-25,0,0
4,5,NaN,Afghanistan,2020-01-26,0,0
...,...,...,...,...,...,...
22639,32705,NaN,Zimbabwe,2020-03-31,8,1
22640,32706,NaN,Zimbabwe,2020-04-01,8,1
22641,32707,NaN,Zimbabwe,2020-04-02,9,1
22642,32708,NaN,Zimbabwe,2020-04-03,9,1


In [52]:
print(data['Province_State'].describe())
data.groupby('Province_State')['ConfirmedCases'].sum()

count         9620
unique         130
top       Zhejiang
freq            74
Name: Province_State, dtype: object


Province_State
Alabama     11316
Alaska       1310
Alberta      9429
Anguilla       19
Anhui       59527
            ...  
Wyoming      1344
Xinjiang     4435
Yukon          46
Yunnan      10979
Zhejiang    76233
Name: ConfirmedCases, Length: 130, dtype: int64

more than half province state is NA, dropping province state sounds ok, we can link population density later if needed.

In [53]:
#data.drop('Province_State', axis=1, inplace=True)
data.drop('Id', axis=1, inplace=True)

### EDA

In [54]:
cases = data.groupby('Country_Region')['ConfirmedCases'].sum()
ind = np.argsort(cases)[::-1]
cases = cases[ind]
deaths = data.groupby('Country_Region')['Fatalities'].sum()

In [55]:
fatality_rate_bycountry = deaths / cases

In [56]:
indf = np.argsort(fatality_rate_bycountry)[::-1]
fatality_rate_bycountry = fatality_rate_bycountry[indf]

In [57]:
top_20_fatal = fatality_rate_bycountry[:20]
top_20_cases = cases[:20]

In [58]:
top_20_fatal

Country_Region
Sudan               0.329545
Gambia              0.254902
Botswana            0.217391
MS Zaandam          0.181818
Angola              0.170732
Nicaragua           0.169811
Cabo Verde          0.162162
Guyana              0.155556
Zimbabwe            0.147727
Syria               0.118182
Italy               0.103274
San Marino          0.098000
Gabon               0.095808
Bangladesh          0.095299
Indonesia           0.086493
Mauritania          0.085714
Spain               0.080929
Congo (Kinshasa)    0.080136
Algeria             0.077397
Iraq                0.075781
dtype: float64

In [59]:
top_20_cases

Country_Region
China             4516642
US                2122889
Italy             1681364
Spain             1204247
Germany            938675
Iran               734551
France             684540
United Kingdom     310944
Korea, South       297163
Switzerland        231843
Netherlands        163044
Belgium            153548
Turkey             137445
Austria            129966
Canada              98284
Portugal            84614
Norway              72401
Brazil              71428
Sweden              67010
Israel              60914
Name: ConfirmedCases, dtype: int64

Aside from Spain and Italy, top 20 fatalaties do not have the highest number of cases, in fact it seems African/Developed coutries, related to health care system? most likely, 
we can check this dataset later: https://www.kaggle.com/danevans/world-bank-wdi-212-health-systems

other factor could be pollution, China has considerably higher pollution rate than most countries but health care and tech advance plays a huge role clearly. We'll see...

## Data Featuring

In [60]:
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['weekday'] = data['Date'].dt.weekday

weekday_name = data['Date'].dt.weekday_name  # for reference if needed

In [61]:
#data.drop('Date', axis=1, inplace=True)

In [62]:
cases, fatalities = data['ConfirmedCases'], data['Fatalities']  # Get y_1, y_2
data.drop('ConfirmedCases', axis=1, inplace=True)
data.drop('Fatalities', axis=1, inplace=True)

### Join density data
source = https://worldpopulationreview.com/countries/countries-by-density/#dataTable

In [63]:
dens = {}
with open('data/rho.json', 'r') as js:
    d = json.load(js)['data']
for js in d:
    dens[js['name']] = js['density']  ## /km2

In [64]:
data['Density'] = data['Country_Region'].apply(lambda c: float(dens[c]) if c in dens else float('nan'))

### Join health care data

In [65]:
health_care = pd.read_csv(path+'../2.12_Health_systems.csv')

In [66]:
missing_countries = set(data['Country_Region']) - set(health_care['Country_Region'])
print ("Missing countries are: {}".format(missing_countries))

Missing countries are: {'Kosovo', 'Sierra Leone', 'Holy See', 'West Bank and Gaza', 'Taiwan*', 'Diamond Princess', 'MS Zaandam', 'Burma'}


In [67]:
countries = set(data['Country_Region'])
health_js = {}
for i, rec in health_care.iterrows():
    if rec['Country_Region'] in countries:
        health_js[rec['Country_Region']] = [
            rec['Health_exp_pct_GDP_2016'], rec['Physicians_per_1000_2009-18'],
            rec['Completeness_of_death_reg_2008-16']
        ]

data['Health_GDP'] = data['Country_Region'].apply(
    lambda c: health_js[c][0] if c in health_js else float('nan'))
data['Physicians'] = data['Country_Region'].apply(
    lambda c: health_js[c][1] if c in health_js else float('nan'))
data['Completeness_of_death_reg'] = data['Country_Region'].apply(
    lambda c: health_js[c][2] if c in health_js else float('nan'))

we just need to fill Density, Health GDP, and physicians with mean from other countries and create new columns

In [68]:

for column in data:
    print(column)
    print(
        f"\t\t dtype: {data[column].dtype}, # NA's{data[column].isnull().sum()}"
    )
    print()

Province_State
		 dtype: category, # NA's13024

Country_Region
		 dtype: category, # NA's0

Date
		 dtype: datetime64[ns], # NA's0

Year
		 dtype: int64, # NA's0

Month
		 dtype: int64, # NA's0

Day
		 dtype: int64, # NA's0

weekday
		 dtype: int64, # NA's0

Density
		 dtype: float64, # NA's5106

Health_GDP
		 dtype: float64, # NA's8584

Physicians
		 dtype: float64, # NA's8510

Completeness_of_death_reg
		 dtype: float64, # NA's13098



In [69]:
for column in ['Density', 'Health_GDP', 'Health_GDP', 'Completeness_of_death_reg']:
    data[column+'_NA'] = data[column].isnull()
    data[column] = data[column].fillna(data[column].mean())

In [70]:
data.drop('Date', axis=1, inplace=True)

In [71]:
data.head()

,Province_State,Country_Region,Year,Month,Day,weekday,Density,Health_GDP,Physicians,Completeness_of_death_reg,Density_NA,Health_GDP_NA,Completeness_of_death_reg_NA
0,NaN,Afghanistan,2020,1,22,2,59.685,10.2,0.3,91.907752,False,False,True
1,NaN,Afghanistan,2020,1,23,3,59.685,10.2,0.3,91.907752,False,False,True
2,NaN,Afghanistan,2020,1,24,4,59.685,10.2,0.3,91.907752,False,False,True
3,NaN,Afghanistan,2020,1,25,5,59.685,10.2,0.3,91.907752,False,False,True
4,NaN,Afghanistan,2020,1,26,6,59.685,10.2,0.3,91.907752,False,False,True


In [72]:
data['ConfirmedCases'], data['Fatalities'] = cases, fatalities

In [73]:
data.to_feather('data/covid19_data')

In [345]:
x = data[['ConfirmedCases', 'Fatalities', 'Density', 'Health_GDP', 'Health_GDP', 'Completeness_of_death_reg']]